# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw2_123456789_987654321.ipynb` if you submitted in pairs and `hw2_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**
1. Your code must run without errors. Make sure your `numpy` version is at least 1.15.4 and that you are using at least python 3.6. Changes of the configuration we provided are at your own risk. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Answers to qualitative questions should be written in **markdown** cells (with $\LaTeX$ support). Answers that will be written in commented code blocks will not be checked.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

# I have read and understood the instructions: 201466349, 313591851

In [773]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Ignore warnings
import warnings

warnings.filterwarnings('ignore')

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [774]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)

    def __repr__(self):
        return str(self.data)

In [775]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children

[6, 7]

## Data preprocessing

For the following exercise, we will use a dataset containing mushroom data `agaricus-lepiota.csv`. 

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous
one (=there are only two classes **edible** and **poisonous**). 
    
The dataset contains 8124 observations with 22 features:
1. cap-shape: bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
4. bruises: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g,green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [776]:
# load dataset
data = pd.read_csv('agaricus-lepiota.csv')

One of the advantages of the Decision Tree algorithm is that almost no preprocessing is required. However, finding missing values is always required.

In [777]:
#############################################################################
# TODO: Find columns with missing values and remove them from the data.#
#############################################################################
data = data.dropna(axis=1)
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

We will split the dataset to `Training` and `Testing` datasets.

In [778]:
from sklearn.model_selection import train_test_split

# Making sure the last column will hold the labels
X, y = data.drop('class', axis=1), data['class']
X = np.column_stack([X, y])
# split dataset using random_state to get the same split each time
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

Training dataset shape:  (6093, 22)
Testing dataset shape:  (2031, 22)


In [779]:
y.shape

(8124,)

## Impurity Measures

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` and `calc_entropy`. You are encouraged to test your implementation (10 points).

In [780]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
 
    Returns the gini impurity.    
    """
    gini = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    y = data if data.ndim == 1 else data[:, -1]
    s = len(y)
    s_i = np.unique(y, return_counts=True)[1]
    gini = 1 - np.sum((s_i / s) ** 2)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return gini

In [781]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """
    entropy = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    y = data if data.ndim == 1 else data[:, -1]
    s = len(y)
    s_i = np.unique(y, return_counts=True)[1]
    entropy = np.sum(-(s_i / s) * np.log2(s_i / s))
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return entropy

In [782]:
##### Your Tests Here #####
eps = 10 ** -6
assert np.abs(calc_entropy(X[:3, :]) - 0.9182958340544896) < eps
assert np.abs(calc_gini(X[:3, :]) - 4 / 9) < eps

## Goodness of Split

Given a feature the Goodnees of Split measures the reduction in the impurity if we split the data according to the feature.
$$
\Delta\varphi(S, A) = \varphi(S) - \sum_{v\in Values(A)} \frac{|S_v|}{|S|}\varphi(S_v)
$$

In our implementation the goodness_of_split function will return either the Goodness of Split or the Gain Ratio as learned in class. You'll control the return value with the `gain_ratio` parameter. If this parameter will set to False (the default value) it will return the regular Goodness of Split. If it will set to True it will return the Gain Ratio.
$$
GainRatio(S,A)=\frac{InformationGain(S,A)}{SplitInformation(S,A)}
$$
Where:
$$
InformationGain(S,A)=Goodness\ of\ Split\ calculated\ with\ Entropy\ as\ the\ Impurity\ function \\
SplitInformation(S,A)=- \sum_{a\in A} \frac{|S_a|}{|S|}\log\frac{|S_a|}{|S|}
$$
NOTE: you can add more parameters to the function and you can also add more returning variables (The given parameters and the given returning variable should not be touch). (10 Points)

In [783]:
def goodness_of_split(data, feature, impurity_func, gain_ratio=False):
    """
    Calculate the goodness of split of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.
    - gain_ratio: goodness of split or gain ratio flag.

    Returns the goodness of split (or the Gain Ration).  
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    split_information = 1
    if gain_ratio:
        impurity_func = calc_entropy  # Forcing the impurity to be entropy
        # Calculate the split_info is similar to calculating "entropy"
        # of the feature col, using the feature values as "classes"
        split_information = calc_entropy(data[:, feature])

    data = pd.DataFrame(data)

    phi_S = impurity_func(data.values)

    Sv = data[feature].value_counts()
    S = data.shape[0]
    w_Sv = Sv / S  # the weight of each feature value v in the wighted-sum

    # next we split the dataframe into k parts, and calculate each part's impurity w.r. to c
    C = data.columns[-1]
    phi_Sv = data.groupby(feature).agg(
        {C: lambda x: impurity_func(x.values)}
    )

    # we match the weights of v in `feature` to the right impurity value
    merged = pd.merge(w_Sv, phi_Sv, left_index=True, right_index=True)

    goodness = phi_S - np.dot(merged.iloc[:, 0].values, merged.iloc[:, 1])
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return goodness / split_information

## Building a Decision Tree

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes categorical data. In this exercise, when splitting a node create the number of children needed according to the attribute unique values.

Complete the class `DecisionNode`. The structure of this class is entirely up to you. 

Complete the function `build_tree`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points)

In [784]:
class DecisionNode:
    """
    This class will hold everything you require to construct a decision tree.
    The structure of this class is up to you. However, you need to support basic 
    functionality as described above. It is highly recommended that you 
    first read and understand the entire exercise before diving into this class.
    """

    def __init__(self, remaining_features, data, impurity, gain_ratio, min_samples_split, max_depth, feature_val=None):
        data_df = pd.DataFrame(data)
        self.__feature_val = feature_val  # the value of the ancestor's feature split. None for root
        self.__most_common_label = data_df.iloc[:, -1].mode()[0]  # return the most common label in `data`
        self.__is_leaf = self.is_pure(data) or self.reached_max_depth(max_depth) or self.reached_min_sample_split(data, min_samples_split)

        if self.__is_leaf:
            self.__labels_count = self.get_labels_count(data)
        else:
            self.__children = []
            self.__feature, remaining_features = self.get_best_split(data, remaining_features, impurity, gain_ratio)
            for f_val in data_df[self.__feature].unique():
                self.add_child(
                    node=DecisionNode(
                        feature_val=(self.__feature, f_val),
                        remaining_features=remaining_features,
                        data=data_df.loc[data_df[self.__feature] == f_val].values,
                        impurity=impurity,
                        gain_ratio=gain_ratio,
                        min_samples_split=min_samples_split,
                        max_depth=max_depth - 1
                    )
                )

    @property
    def is_leaf(self):
        return self.__is_leaf

    @property
    def children(self):
        return self.__children

    def add_child(self, node):
        self.__children.append(node)

    def predict(self, instance):
        if self.__is_leaf:
            return self.__most_common_label
        else:
            f_val = instance[self.__feature]
            for child in self.__children:
                if child.__feature_val[1] == f_val:
                    return child.predict(instance)
        return self.__most_common_label

    def __repr__(self):
        if self.__is_leaf:
            return f'[X{self.__feature_val[0]}={self.__feature_val[1]}, leaf]: [{self.__labels_count}]'
        if not self.__feature_val:
            return f'[ROOT,feature=X{self.__feature}]'
        return f'[X{self.__feature_val[0]}={self.__feature_val[1]}, feature=X{self.__feature}]'

    @staticmethod
    def get_best_split(data, remaining_features, impurity, gain_ratio):
        remaining_features = remaining_features.copy()
        best_split = (-1, -1)
        for feature in remaining_features:
            split_score = goodness_of_split(
                data=data,
                feature=feature,
                impurity_func=impurity,
                gain_ratio=gain_ratio
            )
            if split_score > best_split[1]:
                best_split = (feature, split_score)

        feature = best_split[0]
        remaining_features.remove(feature)
        return feature, remaining_features

    @staticmethod
    def is_pure(data):
        return len(np.unique(data[:, -1])) == 1

    @staticmethod
    def reached_max_depth(max_depth):
        return max_depth == 0

    @staticmethod
    def reached_min_sample_split(data, min_samples_split):
        return data.shape[0] < min_samples_split

    @staticmethod
    def get_labels_count(data):
        labels, counts = np.unique(data[:, -1], return_counts=True)
        return {labels[i]: counts[i] for i in range(len(labels))}

In [785]:
def build_tree(data, impurity, gain_ratio=False, min_samples_split=1, max_depth=1000):
    """
    Build a tree using the given impurity measure and training dataset. 
    You are required to fully grow the tree until all leaves are pure. 

    Input:
    - data: the training dataset.
    - impurity: the chosen impurity measure. Notice that you can send a function
                as an argument in python.
    - gain_ratio: goodness of split or gain ratio flag
    - min_samples_split: the minimum number of samples required to split an internal node
    - max_depth: the allowable depth of the tree

    Output: the root node of the tree.
    """
    root = None
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    root = DecisionNode(
        remaining_features=list(range(data.shape[1] - 1)),  # removing last column from feature list
        data=data,
        impurity=impurity,
        gain_ratio=gain_ratio,
        min_samples_split=min_samples_split,
        max_depth=max_depth
    )
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return root

In [786]:
%%time
# python supports passing a function as an argument to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini)  # gini and goodness of split
tree_entropy = build_tree(data=X_train, impurity=calc_entropy)  # entropy and goodness of split
tree_entropy_gain_ratio = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True)  # entropy and gain ratio

CPU times: user 3min 14s, sys: 2.13 s, total: 3min 16s
Wall time: 3min 18s


## Tree evaluation

Complete the functions `predict` and `calc_accuracy`. (10 points)

In [787]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree
 
    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. Note that the last element 
                of this vector is the label of the instance.
 
    Output: the prediction of the instance.
    """
    return node.predict(instance)

In [788]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree
 
    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated
 
    Output: the accuracy of the decision tree on the given dataset (%).
    """
    predictions = np.apply_along_axis(node.predict, 1, dataset)
    y = dataset[:, -1]
    correct_predictions = np.count_nonzero(predictions == y)
    accuracy = correct_predictions / len(y)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy

After building the three trees using the training set, you should calculate the accuracy on the test set. For each tree print the training and test accuracy. Select the tree that gave you the best test accuracy. For the rest of the exercise, use that tree (when you asked to build another tree use the same impurity function and same gain_ratio flag).

In [789]:
def print_scores(nodes: dict):
    for name, node in nodes.items():
        print(f'{name} train accuracy: ', calc_accuracy(node, X_train))
        print(f'{name} test accuracy: ', calc_accuracy(node, X_test), '\n')

In [790]:
print_scores({
    'tree_gini': tree_gini,
    'tree_entropy': tree_entropy,
    'tree_entropy_gain_ratio': tree_entropy_gain_ratio
})

tree_gini train accuracy:  1.0
tree_gini test accuracy:  0.7730182176267848 

tree_entropy train accuracy:  1.0
tree_entropy test accuracy:  0.7715411127523387 

tree_entropy_gain_ratio train accuracy:  1.0
tree_entropy_gain_ratio test accuracy:  0.7853274249138356 



tree_entropy_gain_ratio is the best tree

## Depth pruning

(15 points)

Consider the following max_depth values: [1, 2, 3, 4, 5, 6, 7, 8]. For each value, construct a tree and prune it according to the max_depth value = don't let the tree to grow beyond this depth. Next, calculate the training and testing accuracy.<br>
On a single plot, draw the training and testing accuracy as a function of the max_depth. Mark the best result on the graph with red circle.

In [ ]:
#### Your code here ####
train_acc = []
test_acc = []
for max_depth in [1, 2, 3, 4, 5, 6, 7, 8]:
    tree = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, max_depth=max_depth)
    train_acc.append(calc_accuracy(tree, X_train))
    test_acc.append(calc_accuracy(tree, X_test))

In [ ]:
def plot_scores(train_acc, test_acc, hyperparam, x, sizes=None):
    import seaborn as sns
    fig, ax = plt.subplots(1, 1, figsize=(12,7))
    sns.lineplot(y=train_acc, x=x, ax=ax, label='train')
    sns.lineplot(y=test_acc, x=x, ax=ax, label='test')
    if sizes:
        ax2 = ax.twinx()
        sns.scatterplot(y=sizes, x=x, ax=ax2, label='train tree size', color='red', legend=False)
        ax2.legend(loc='lower left')
    ax.set(title=f'train/test accuracy by {hyperparam} tunning', xlabel=hyperparam, ylabel='accuracy')
    ax.scatter(x[test_acc.index(max(test_acc))], max(test_acc), marker="o", c="red", s=400, lw=4, alpha=0.1)
    # ax.legend();

In [ ]:
plot_scores(train_acc, test_acc, 'max_depth', x=np.arange(1,9))

## Min Samples Split

(15 points)

Consider the following min_samples_split values: [1, 5, 10, 20, 50]. For each value, construct a tree and prune it according to the min_samples_split value = don't split a node if the number of sample in it is less or equal to the min_samples_split value. Next, calculate the training and testing accuracy.<br>
On a single plot, draw the training and testing accuracy as a function of the min_samples_split. Mark the best result on the graph with red circle. (make sure that the x-axis ticks represent the values of min_samples_split)

In [ ]:
#### Your code here ####
train_acc = []
test_acc = []
for min_samples_split in [1, 5, 10, 20, 50]:
    tree = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, min_samples_split=min_samples_split)
    train_acc.append(calc_accuracy(tree, X_train))
    test_acc.append(calc_accuracy(tree, X_test))

In [ ]:
plot_scores(train_acc, test_acc, 'min_samples_split', x=[1, 5, 10, 20, 50])

Build the best 2 trees:
1. tree_max_depth - the best tree according to max_depth pruning
1. tree_min_samples_split - the best tree according to min_samples_split pruning

In [ ]:
#### Your code here ####
tree_max_depth = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, max_depth=4)
tree_min_samples_split = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, min_samples_split=50)

## Number of Nodes

(5 points)

Complete the function counts_nodes and print the number of nodes in each tree and print the number of nodes of the two trees above

In [ ]:
def count_nodes(node: DecisionNode):
    """
    Count the number of node in a given tree
 
    Input:
    - node: a node in the decision tree.
 
    Output: the number of node in the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if node.is_leaf:
        return 1
    else:
        return 1 + sum([count_nodes(c) for c in node.children])
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################


In [ ]:
print('tree_max_depth node count:', count_nodes(tree_max_depth))

In [ ]:
print('tree_min_samples_split node count', count_nodes(tree_min_samples_split))

## Print the tree

Complete the function `print_tree`. Your tree should be visualized clearly. You can use the following example as a reference:
```
[ROOT, feature=X0],
  [X0=a, feature=X2]
    [X2=c, leaf]: [{1.0: 10}]
    [X2=d, leaf]: [{0.0: 10}]
  [X0=y, feature=X5], 
    [X5=a, leaf]: [{1.0: 5}]
    [X5=s, leaf]: [{0.0: 10}]
  [X0=e, leaf]: [{0.0: 25, 1.0: 50}]
```
In each brackets:
* The first argument is the parent feature with the value that led to current node
* The second argument is the selected feature of the current node
* If the current node is a leaf, you need to print also the labels and their counts

(5 points)

In [ ]:
# you can change the function signeture
def print_tree(node: DecisionNode, tab=0):
    '''
    prints the tree according to the example above

    Input:
    - node: a node in the decision tree

    This function has no return value
    '''
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if node.is_leaf:
        print(f'{"  " * tab}{node}')
    else:
        print(f'{"  " * tab}{node}')
        for child in node.children:
            print_tree(child, tab=tab+1)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

print the tree with the best test accuracy and with less than 50 nodes (from the two pruning methods)

In [ ]:
train_acc = []
test_acc = []
sizes = []
for min_samples_split in np.arange(150, 350, 25):
    # for max_depth in [2, 3, 4, 5, 6, 7, 8, 9]:
    tree = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, min_samples_split=min_samples_split, max_depth=4)
    train_acc.append(calc_accuracy(tree, X_train))
    test_acc.append(calc_accuracy(tree, X_test))
    sizes.append(count_nodes(tree))

In [ ]:
plot_scores(train_acc, test_acc, 'min_samples_split', x=np.arange(150, 350, 25), sizes=sizes)

In [ ]:
tree = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True, min_samples_split=225, max_depth=4)
count_nodes(tree)

In [ ]:
print_tree(tree)